In [1]:
import pandas as pd
import numpy as np
import os
import random
import re
import torch
import copy

def flat(input_lst):
    lst= []
    for i in input_lst:
        if type(i) is list:
            for j in i:
                lst.append(j)
        else:
            lst.append(i)
    return(lst)

In [2]:
root = os.path.join(os.path.dirname(os.getcwd()), 'Package2task_dis_V10')

In [4]:
def process_dfs(file_pth, root):
    df_ = pd.read_csv(os.path.join(root, file_pth), header=None) # pay attention to the file choosing, file_1/2/3

    products_ = []
    task_ = []
    spatial_ = []
    for i, row in df_.iterrows():
        task_.append(list(row)[0].lower())
        products_.append(list(set([r.lower() for r in list(row)[1:] if (not str(r)=='nan' and not 'contain' in r and not 'sp-' in r)])))
        spatial_.append(flat([r.lower().strip().split('-')[1].split(';') for r in list(row) if 'sp-' in str(r)]))

    return df_, products_, task_, spatial_

def write_data(file_name, products_, tasks_, spatial_, task_records, spatial_included, products_num=None, spatial_only=False):
    sp_dir = {}
    data = []
    task2id = {}
    with open(file_name, mode='w') as f:
        for i, t in enumerate(task_):
            task2id.update({t.strip():i})
            origin_products = products_[i] # get the original products for random selection
            for j in range(task_records): # the number of training records for a single task type
                f.write(t.strip() + '--')
                temp_train_products = []
                if products_num==None:
                    products_num = random.randint(2*len(origin_products), 4*len(origin_products))
                start = 1
                while(start <= products_num): # the number of products for a single data sample for the current task
                    temp_train_products.append(random.choice(origin_products))
                    start += 1
                final_temp_products = temp_train_products
                # print('final_temp_products, ', len(final_temp_products))

                if spatial_included:
                    temp_train_products_spatial = [] #copy.deepcopy(temp_train_products)
                    current_spatials = spatial_[i] # get the original spatial information
                    sp_dir.update({task_[i] : current_spatials})
                    for sp in current_spatials:
                        for tp in temp_train_products:
                            sp_data = sp + ' contains ' + tp
                            temp_train_products_spatial.append(sp_data)
                    
                    if spatial_only==True:
                        final_temp_products = temp_train_products_spatial
                    else:
                        final_temp_products = temp_train_products + temp_train_products_spatial

                for p in final_temp_products:
                    f.write(p + ', ')
                f.write('\n')
                data.append((t.strip(), final_temp_products))
    f.close()
    return data, task2id
    

### generate training or testing data (detail granularity)

In [7]:
file_detail = 'data_separated_detail.csv'
#file_out_name = 'test_spatial_data_detail.txt' 
file_out_name = 'train_data_detail.txt'# train_data_detail.txt   train_spatial_data_detail.txt
file_out_name = os.path.join(root, file_out_name)
print(file_out_name)

spatial_included = False # True False
spatial_only = False # True False

df_, products_, task_, spatial_ = process_dfs(file_detail, root)
data, task2id = write_data(file_out_name, products_, task_, spatial_, task_records=800, products_num=25, spatial_included=spatial_included, spatial_only=spatial_only)
df_.head(6)


C:\JupyterFiles\Package2task_dis_V10\train_data_detail.txt


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Board pretreatment and punching,sp-none,Lysaght Bondek,Gemtree FRP Board,Gypsum Board,Cement Board,Knauf FRP Board,Promatect HBoard,NaN,NaN,NaN,NaN,NaN
1,2D panel assembly,sp-plane,Corrugated profile,Composite floor slab,Shear stud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Butt weld and fillet weld,sp-none,metal pencil,Welding Electrode,Electrode,Welding rod,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3D assembly,sp-room space,Steel ceiling beam,Steel Purlins,Steel floor beam,Corner Casting,Steel Angle,Steel Column,Corner Casting,NaN,NaN,NaN,NaN
4,Apply fire paint at strucutral member,sp-room space,Fire protection paint,Firecut FM-900 paint,Firecut paint,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Rebar Fixing,sp-none,Reinforcement Bar,Bar Chair,Steel Reinforcement Bar,Rebar,Tie Wire,Spacer,NaN,NaN,NaN,NaN,NaN


### generate testing data (abstract granularity)

In [5]:
task2id

{'board pretreatment and punching': 0,
 '2d panel assembly': 1,
 'butt weld and fillet weld': 2,
 '3d assembly': 3,
 'apply fire paint at strucutral member': 4,
 'rebar fixing': 5,
 'pouring concrete': 6,
 'door ironmongery installation': 7,
 'window ironmongery installation': 8,
 'door frame installation': 9,
 'window frame installation': 10,
 'installation of gypsum board at wall': 11,
 'installation of mep conduit at wall': 12,
 'stud installation at wall': 13,
 'rockwool in-fill at wall': 14,
 'fire board installation at wall': 15,
 'stud installation at ceiling': 16,
 'rockwool in-fill at ceiling': 17,
 'fire board installation at ceiling': 18,
 'layer and coat painting at wall': 19,
 'layer and coat painting at ceiling': 20,
 'installation of lighting': 21,
 'electrical cable wiring': 22,
 'installation of mep trunking at ceiling': 23,
 'installation of sprinkler pipes': 24,
 'window type ac installation': 25,
 'installation of vinyl floor and skirting': 26,
 'cabinets installati

In [6]:
file_abs = 'data_separated_abs.csv'
file_out_name = 'test_spatial_data_abs.txt'
#file_out_name = 'test_data_abs.txt'         
print(file_out_name)

ranges = {'board pretreatment and punching':[0], 'assembly weld and fire paint':list(range(1,5)), 'rebar and concrete':list(range(5,7)),
          'door and window installation':list(range(7,11)),'install and paint wall and ceiling stud rockwool and fire board':list(range(11,21)), 
          'install lighting cable and mep pipes':list(range(21,26)),'install floor and cabinets':list(range(26,28))}

with open(os.path.join(root, file_out_name), mode='w') as f:
    for i, d in enumerate(data):
        t_name = d[0]
        t_data = d[1]
        t_id = task2id[t_name]
        print(t_name, ' ', t_id)
        for t_key, ran in ranges.items():
            if t_id in ran:
                t_name = str(t_key)
        print(t_name, ' ', t_id)
        f.write(t_name.strip() + '-- ')
        for td in t_data:
            f.write(td + ', ')
        f.write('\n')
f.close()

#write_data(file_out_name, products_, task_, spatial_, task_records=100, products_num=30, spatial_included=spatial_included, spatial_only=spatial_only)
#df_.head(6)
         

test_spatial_data_abs.txt
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
board pretreatment and punching   0
bo

rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5
rebar and concrete   5
rebar fixing   5


install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
install and paint wall and ceiling stud rockwool and fire board   11
installation of gypsum board at wall   11
i

rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool and fire board   17
rockwool in-fill at ceiling   17
install and paint wall and ceiling stud rockwool 

installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   23
install lighting cable and mep pipes   23
installation of mep trunking at ceiling   2

### generate training data (medium granularity)

In [23]:
# please note train data is at the medium granularity (i.e., using the data_separated_med.csv file)
file_med = '_med.csv'
#file_out_name = 'train_spatial_data.txt'
#file_out_name = 'train_data.txt'
#print(file_out_name)

df_, products_, task_, spatial_ = process_dfs(file_med)
#write_data(file_out_name, products_, task_, spatial_, task_records=900, spatial_included=spatial_included, spatial_only=spatial_only)
df_.head(6)


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,Board pretreatment and punching,sp-none,Lysaght Bondek,Gemtree FRP Board,Gypsum Board,Cement Board,Knauf FRP Board,Promatect HBoard,none contains gemtree FRP board,none contains gypsum board,...,Cement Board,Knauf FRP Board,Gemtree FRP Board,Gypsum Board,Cement Board,Knauf FRP Board,Lysaght Bondek,NaN,NaN,NaN
1,2D panel assembly,sp-plane,Corrugated profile,Corrugated profile,Composite floor slab,Shear stud,ceil system contains corrugated profile,plane contains shear stud,wall system contains corrugated profile,Composite floor slab,...,plane contains shear stud,wall system contains corrugated profile,Composite floor slab,wall system contains corrugated profile,NaN,NaN,NaN,NaN,NaN,NaN
2,Butt weld and fillet weld,sp-none,Welding Electrode,Electrode,Welding Electrode,Welding Electrode,Electrode,filler rod,metal pencil,Welding Electrode,...,Electrode,Welding Electrode,Welding Electrode,Electrode,Welding Electrode,Electrode,Welding rod,NaN,NaN,NaN
3,3D assembly,sp-room space,2D panel,Corner Casting,Steel Column,Steel Angle,Shear stud,CandleLoc System,Steel ceiling beam,Steel Purlins,...,Corner Casting,Steel Column,Steel Angle,Corner Casting,Steel Column,Steel Angle,NaN,NaN,NaN,NaN
4,Apply fire paint at strucutral member,sp-room space,3D steel frame,Fire protection paint,Firecut FM-900 paint,Firecut paint,Fire protection paint,Firecut FM-900 paint,Firecut paint,Fire protection paint,...,Fire protection paint,Firecut FM-900 paint,Firecut paint,Firecut paint,Fire protection paint,Firecut FM-900 paint,Firecut paint,NaN,NaN,NaN
5,Rebar Fixing,sp-none,Spacer,Tie Wire,Steel Reinforcement Bar,Rebar,Reinforcement Bar,Bar Chair,Steel Reinforcement Bar,Rebar,...,Steel Reinforcement Bar,Rebar,Reinforcement Bar,Bar Chair,Steel Reinforcement Bar,Rebar,Reinforcement Bar,Bar Chair,NaN,NaN


### generate testing data of medium granularity

In [18]:
sp_path = os.path.join(root, 'test_data.txt')
test_spatial_path = os.path.join(root, 'test_spatial_data.txt')

with open(sp_path, mode='r', encoding='UTF-8') as f:
    for line in f:
        test_noisy_products = []
        task = line.split('--')[0].strip().lower()
        products = [p.strip() for p in line.split('--')[1].split(',')]

        test_noisy_products = copy.deepcopy(products) # record original products
        sp_info = sp_dir[task]
        for sp in sp_info: # add spatial info
            for tp in products:
                sp_data = sp + ' contains ' + tp
                test_noisy_products.append(sp_data)
        #print(test_noisy_products)
        with open(test_spatial_path, mode='a', encoding='UTF-8') as f:
            f.write(task + '-- ')
            for d in test_noisy_products:
                f.write(d + ', ')
            f.write('\n')
f.close()

### generate testing data for detail granularity

In [23]:
if spatial_included == True:
    sp_dir_name = 'sp_dir' + file_2 # file_2 file_1 file_3
sp_dir_name

'sp_dir_detail.csv'

### test reading train data

In [115]:
data_path = os.path.join(root, 'train_data.txt')
task_labels = []
products = []
all_terms = []

with open(data_path, mode='r') as f:
    for line in f:
        temp_task = line.strip().split('--')[0]
        temp_task = re.sub(r'[^ a-zA-Z0-9]', '', temp_task)
        temp_task = re.sub(r'\s+', ' ', temp_task)
        task_labels.append(temp_task)
        
        temp_products = [p.strip() for p in line.strip().split('--')[1].split(',') if not p=='']
        for i, _ in enumerate(temp_products):
            temp_products[i] = re.sub(r'[^ a-zA-Z0-9]', '', temp_products[i])
            temp_products[i] = re.sub(r'\s+', ' ', temp_products[i])
        products.append(temp_products)
    all_terms.extend(task_labels)
    all_terms.extend(products)
    
    products_data = np.asarray(products)
    label_data = np.asarray([task_labels])
    all_terms = list(set(list(flat(all_terms))))

f.close()


[['board pretreatment and punching' 'board pretreatment and punching'
  'board pretreatment and punching' ... 'cabinets installation'
  'cabinets installation' 'cabinets installation']]
[list(['gypsum board', 'cement board', 'promatect hboard', 'promatect hboard', 'gypsum board'])
 list(['promatect hboard', 'cement board', 'lysaght bondek', 'lysaght bondek'])
 list(['lysaght bondek', 'gemtree frp board', 'lysaght bondek', 'lysaght bondek', 'cement board', 'promatect hboard'])
 ...
 list(['toilet cabinet', 'cabinets', 'wall cabinet', 'toiletsink cabinet', 'tv cabinet'])
 list(['toilet cabinet', 'toiletsink cabinet', 'tv cabinet', 'tv cabinet'])
 list(['sink cabinet', 'toilet cabinet', 'toilet cabinet'])]
